In [ ]:
pip install neo4j langchain langchain_community openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

In [ ]:
from neo4j import GraphDatabase

NEO4J_URI="neo4j+s://fca10099.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="iqb2q2po8SD6bubpHHHUxFwhIzLqmTzX2MDgewLBjYg"

graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

print(graph.schema)

Node properties:
Prostate_Cancer {name: STRING}
Medications {name: STRING}
Diagnostic_Tests {name: STRING}
Side_Effects {name: STRING}
Relationship properties:

The relationships:
(:Prostate_Cancer)-[:is_treated_by]->(:Medications)
(:Prostate_Cancer)-[:is_diagnosed_by]->(:Diagnostic_Tests)
(:Medications)-[:has_side_effect]->(:Side_Effects)
(:Medications)-[:is_treated_by]->(:Prostate_Cancer)


In [ ]:
os.environ['OPENAI_API_KEY'] = API_KEY

In [ ]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

graph_result = chain.run("What are the medications used for treating Prostate cancer?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Prostate_Cancer)-[:is_treated_by]->(m:Medications) RETURN m.name
Full Context:
[{'m.name': 'PARP inhibitors'}, {'m.name': 'androgen receptor pathway inhibitors (ARSi)'}, {'m.name': 'radiation'}, {'m.name': 'radiopharmaceuticals'}, {'m.name': 'chemotherapy'}, {'m.name': 'immunotherapy'}, {'m.name': 'Androgen deprivation therapy (ADT)'}, {'m.name': 'taxane-based chemotherapy'}, {'m.name': 'androgen receptor signaling inhibitors'}, {'m.name': 'Olaparib'}]

> Finished chain.


In [ ]:
graph_result

'PARP inhibitors, androgen receptor pathway inhibitors (ARSi), radiation, radiopharmaceuticals, chemotherapy, immunotherapy, Androgen deprivation therapy (ADT), taxane-based chemotherapy, androgen receptor signaling inhibitors, Olaparib.'